# Daily rise and fall data

In [1]:
import yfinance as yf
import pandas as pd
import time

# Define stock tickers and filenames
stocks = {
    "BABA": "Alibaba.csv",
    "PHG": "Philips.csv",
    "TSLA": "Tesla.csv",
    "AAPL": "Apple.csv"
}

start_date = "2020-01-01"
end_date = "2024-12-31"

for ticker, filename in stocks.items():
    try:
        data = yf.download(ticker, start=start_date, end=end_date)
        data = data[["Open", "Low", "High", "Close", "Volume"]]
        data.to_csv(filename)
        print(f"{ticker} saved to {filename}")
        time.sleep(5)  
    except Exception as e:
        print(f"Failed to download {ticker}: {e}")

YF.download() has changed argument auto_adjust default to True


[*********************100%***********************]  1 of 1 completed


BABA saved to Alibaba.csv


[*********************100%***********************]  1 of 1 completed


PHG saved to Philips.csv


[*********************100%***********************]  1 of 1 completed


TSLA saved to Tesla.csv


[*********************100%***********************]  1 of 1 completed


AAPL saved to Apple.csv


# Data Dictionary
### Past Performance Features
*   **1D_PastChangePct**: The percentage change between the current day's closing price and the closing price from one trading day prior.
*   **20D_PastChangePct**: The percentage change between the current day's closing price and the closing price from twenty trading days prior.

### Technical Indicators
*   **MA5_GT_MA20**: 1 if the current day's MA5 is greater than the MA20, 0 otherwise.
*   **KDJ**:
    *   $\text{RSV}_t = \frac{\text{Close}_t - \text{LowestLow}_t}{\text{HighestHigh}_t - \text{LowestLow}_t} \times 100$
    *   $K_t = \frac{2}{3} K_{t-1} + \frac{1}{3} \text{RSV}_t$
    *   $D_t = \frac{2}{3} D_{t-1} + \frac{1}{3} K_t$
    *   $J_t = 3 \times K_t - 2 \times D_t$

*   **mfi**: Money Flow Index. A technical indicator used to measure the strength of money flowing into or out of a stock, combining price and volume to assess buying and selling pressure.


### Macroeconomic & Cross-Market Data
*   **Bitcoin_Close**: Daily price of Bitcoin.
*   **Gold_Close**: Daily price of Gold.
*   **Interest_Rate**: US interest rate.

# KDJ calculation

In [1]:
import pandas as pd

# Read the original data
df = pd.read_csv(".csv")

# Ensure the 'Date' column is in datetime format
df['Date'] = pd.to_datetime(df['Date'])

# Calculate RSV (Raw Stochastic Value)
low_9 = df['Low'].rolling(window=9).min()
high_9 = df['High'].rolling(window=9).max()
df['RSV'] = (df['Close'] - low_9) / (high_9 - low_9) * 100

# Initialize K, D, J columns
df['K'] = None
df['D'] = None
df['J'] = None

# Set initial values for January 14, 2020
initial_date = pd.Timestamp('2020-01-14')
initial_k = 90.57
initial_d = 88.66
initial_j = 94.39

start_index = df.index[df['Date'] == initial_date].tolist()
if not start_index:
    raise ValueError("Date 2020-01-14 not found")
start_index = start_index[0]

# Fill in the initial values
df.loc[start_index, 'K'] = initial_k
df.loc[start_index, 'D'] = initial_d
df.loc[start_index, 'J'] = initial_j

# Calculate K, D, J row by row, starting from the next row
for i in range(start_index + 1, len(df)):
    rsv = df.loc[i, 'RSV']
    prev_k = df.loc[i - 1, 'K']
    prev_d = df.loc[i - 1, 'D']
    
    if pd.notna(rsv) and pd.notna(prev_k) and pd.notna(prev_d):
        k = (1/3) * rsv + (2/3) * prev_k
        d = (1/3) * k + (2/3) * prev_d
        j = 3 * k - 2 * d
        df.loc[i, 'K'] = k
        df.loc[i, 'D'] = d
        df.loc[i, 'J'] = j

The history saving thread hit an unexpected error (OperationalError('attempt to write a readonly database')).History will not be written to the database.
         Date        RSV          K          D          J
8  2020-01-14  92.450267      90.57      88.66      94.39
9  2020-01-15  73.834727  84.991576  87.437192  80.100343
10 2020-01-16  68.420074  79.467742  84.780708  68.841808
11 2020-01-17  60.754909  73.230131  80.930516   57.82936
12 2020-01-21  98.282531  81.580931  81.147321  82.448151
13 2020-01-22  79.495234  80.885698  81.060113  80.536869


# mfi calculation

In [1]:
import pandas as pd

# Calculate Typical Price
df["Typical_Price"] = (df["High"] + df["Low"] + df["Close"]) / 3

# Calculate Raw Money Flow
df["Raw_MF"] = df["Typical_Price"] * df["Volume"]

# Determine the direction of the money flow
df["TP_diff"] = df["Typical_Price"].diff()

df["Positive_Flow"] = df.apply(
    lambda row: row["Raw_MF"] if row["TP_diff"] > 0 else 0, axis=1
)
df["Negative_Flow"] = df.apply(
    lambda row: row["Raw_MF"] if row["TP_diff"] < 0 else 0, axis=1
)

# 14-day rolling sum
df["PMF"] = df["Positive_Flow"].rolling(window=14).sum()
df["NMF"] = df["Negative_Flow"].rolling(window=14).sum()

# Calculate Money Ratio and MFI
df["Money_Ratio"] = df["PMF"] / df["NMF"]
df["mfi"] = 100 - 100 / (1 + df["Money_Ratio"])

The history saving thread hit an unexpected error (OperationalError('attempt to write a readonly database')).History will not be written to the database.


NameError: name 'df' is not defined

# Others

In [ ]:
import pandas as pd
import numpy as np

# Read the data
df = pd.read_csv('.csv')

# Date processing and sorting
df['Date'] = pd.to_datetime(df['Date'])
df = df.sort_values('Date').reset_index(drop=True)

df['1D_Up'] = (df['1D_ChangePct'] > 0).astype(int)
df['20D_Up'] = (df['20D_ChangePct'] > 0).astype(int)

df['1D_PastChangePct'] = (df['Close'] - df['Close'].shift(1)) / df['Close'].shift(1)
df['20D_PastChangePct'] = (df['Close'] - df['Close'].shift(20)) / df['Close'].shift(20)

df['MA5_GT_MA20'] = (df['MA5'] > df['MA20']).astype(int)


## MACD RSI

In [8]:
import pandas as pd

def calculate_features(df, company_name):
    close = df['close']
    volume = df['volume']

    ema12 = close.ewm(span=12, adjust=False).mean()
    ema26 = close.ewm(span=26, adjust=False).mean()
    macd = ema12 - ema26
    signal = macd.ewm(span=9, adjust=False).mean()
    macd_diff = macd - signal

    delta = close.diff()
    gain = delta.where(delta > 0, 0)
    loss = -delta.where(delta < 0, 0)
    avg_gain = gain.rolling(14).mean()
    avg_loss = loss.rolling(14).mean()
    rs = avg_gain / avg_loss
    rsi = 100 - (100 / (1 + rs))

    ma20 = close.rolling(window=20).mean()
    std20 = close.rolling(window=20).std()
    upper = ma20 + 2 * std20
    lower = ma20 - 2 * std20
    bb_rel_pos = (close - lower) / (upper - lower)

    vol_change = volume.pct_change()

    label = (close.shift(-1) > close).astype(int)

    df['MACD'] = macd
    df['MACD_diff'] = macd_diff
    df['RSI'] = rsi
    df['BB_rel_pos'] = bb_rel_pos
    df['Vol_Change'] = vol_change
    df['Label'] = label
    df['Company'] = company_name

    df.rename(columns={'date': 'Date'}, inplace=True)

    return df[['Date','Company', 'MACD', 'MACD_diff', 'RSI', 'BB_rel_pos', 'Vol_Change']]

file_map = {
    'Tesla': 'Tesla_price.csv',
    'Alibaba': 'Alibaba_price.csv',
    'Apple': 'Apple_price.csv',
    'Philips': 'Philips_price.csv',
}

for name, file in file_map.items():
    df = pd.read_csv(file)
    df.columns = [col.lower() for col in df.columns]
    if 'close' not in df.columns or 'volume' not in df.columns or 'date' not in df.columns:
        print(f"{name} skipped: missing required columns.")
        continue
    df = calculate_features(df, name)
    output_path = f"{name}_features.csv"
    df.to_csv(output_path, index=False)
    print(f"{name} features saved to {output_path}")

Tesla features saved to Tesla_features.csv
Alibaba features saved to Alibaba_features.csv
Apple features saved to Apple_features.csv
Philips features saved to Philips_features.csv
